In [1]:
import os
os.chdir('h:/Reefs')
import json
import pandas as pd

#import packages
#to install gdal is painful
#the recommendation is to pip install whl files
#download file with respect to ur python version
#cp36 refers to python 3.6
# https://www.lfd.uci.edu/~gohlke/pythonlibs/#gdal
from osgeo import ogr,osr

In [2]:
filename='WCMC008_CoralReef2018_Py_v4_1'

#read shape file
infile = ogr.Open(f"{filename}.shp")
layer = infile.GetLayer()

In [3]:
#convert different features into json then nested lists
#since coral shapes are small areas we just take the first coordinate for simplicity
arr=[]
for i in range(layer.GetFeatureCount()):
    feature = layer.GetFeature(i)
    nested=json.loads(feature.ExportToJson())['geometry']['coordinates']
    arr+=nested[0][0]

In [4]:
#create dataframe
df=pd.DataFrame()
df['identifier']=range(len(arr)/2)
df['longitude']=arr[0::2]
df['latitude']=arr[1::2]
df['feature']='FEATURE_REEF'
df['taille']=2

In [5]:
#shrink data precision
df['lon']=df['longitude'].apply(lambda x:round(x))
df['lat']=df['latitude'].apply(lambda x:round(x))
df=df.sort_values(['longitude','latitude'])
df.reset_index(inplace=True,drop=True) 
ind=df[['lon','lat']].drop_duplicates().index
df=df.loc[ind]

In [6]:
#delete useless cols
del df['lon']
del df['lat']
df.to_csv('corals.csv',index=False)